In [2]:
import numpy as np
import matplotlib.pyplot as plt

import matplotlib.style
import matplotlib as mpl
import matplotlib.colors as colors
import matplotlib.cm as cmx
from mpl_toolkits.axes_grid1 import make_axes_locatable

mpl.style.use('classic')

from astropy.table import Table
from astropy import units as u
from astropy.constants import G
import astropy

import mesa_helper as mh
import os
import shutil

%matplotlib inline

In [2]:
# from $SSW/soho/eit/idl/response/eit_bakeouts.pro per Frederic's advice
# the last entry, Feb-March 2025, comes from Kevin Schenk private comm

In [3]:
on = ['23-may-1996 19:21', '23-jun-1996 19:10', '23-jul-1996 15:48', '05-aug-1996 18:50',
        '09-aug-1996 15:05', '06-sep-1996 19:49', '19-dec-1996 13:14', '03-jan-1997 21:50',
        '20-feb-1997 21:57', '21-mar-1997 14:26', '18-jul-1997 13:30', '28-nov-1997 23:00',
        '20-mar-1998 16:35', '10-apr-1998 22:00', '18-jun-1998 21:58', '24-jun-1998 23:10',
        '21-dec-1998 17:26', '04-feb-1999 17:25', '22-may-1999 15:20', '31-may-1999 13:30',
        '08-oct-1999 20:23', '29-oct-1999 21:41', '04-feb-2000 21:00', '17-jun-2000 03:49',
        '08-sep-2000 21:42', '16-feb-2001 22:31', '10-may-2001 22:49', '15-jun-2001 21:00',
        '31-aug-2001 16:00', '03-oct-2001 19:01', '11-jan-2002 23:06', '30-mar-2002 03:30',
        '15-jun-2002 03:36', '30-aug-2002 23:46', '14-nov-2002 19:04', '01-feb-2003 01:44',
        '19-apr-2003 02:00', '25-jun-2003 00:00', '26-sep-2003 00:00', '26-dec-2003 22:18',
        '18-mar-2004 03:00', '15-jun-2004 20:00', '10-sep-2004 22:00', '11-dec-2004 18:47',
        '11-mar-2005 22:21', '06-jun-2005 21:52', '02-sep-2005 21:37', '02-dec-2005 19:41',
        '28-feb-2006 02:53', '27-may-2006 01:45', '19-aug-2006 00:43', '25-nov-2006 23:05',
	'13-feb-2007 22:49', '18-may-2007 17:36', '26-aug-2007 19:37', '19-nov-2007 11:00',
	'23-feb-2008 02:00', '16-may-2008 18:00', '08-aug-2008 10:00', '12-nov-2008 10:00',
	'07-feb-2009 20:00', '02-may-2009 17:00', '04-aug-2009 18:00', '01-nov-2009 16:00',
	'26-apr-2010 01:10', '20-jul-2010 23:30', '09-feb-2011 17:00', '18-jul-2011 15:30',
	'06-jan-2012 02:35', '02-jul-2012 14:15', '21-jun-2013 17:50', '18-dec-2013 13:45',
	'13-jun-2014 03:05', '11-dec-2014 13:20', '04-jun-2015 03:00', '02-dec-2015 18:08',
	'26-may-2016 16:10', '13-may-2017 03:23', '04-may-2018 04:53', '23-apr-2019 16:48',
	'15-apr-2020 16:40', '05-apr-2021 22:32', '31-mar-2022 18:33', '13-mar-2023 00:40',
    '30-apr-2024 15:18', '25-feb-2025 14:16']

off = ['25-may-1996 16:37', '24-jun-1996 18:48', '23-jul-1996 18:26', '06-aug-1996 10:00',
         '12-aug-1996 13:31', '16-sep-1996 15:43', '19-dec-1996 15:49', '10-jan-1997 10:00',
         '21-feb-1997 11:00', '24-mar-1997 12:56', '27-jul-1997 20:00', '02-dec-1997 03:00',
         '23-mar-1998 16:58', '13-apr-1998 22:00', '22-jun-1998 07:51', '13-oct-1998 20:58',
         '02-feb-1999 20:57', '05-feb-1999 10:45', '24-may-1999 05:00', '01-jun-1999 08:00',
         '12-oct-1999 08:00', '01-nov-1999 15:00', '06-feb-2000 10:00', '20-jun-2000 08:00',
         '11-sep-2000 03:19', '18-feb-2001 10:59', '14-may-2001 11:00', '18-jun-2001 13:00',
         '04-sep-2001 10:00', '05-oct-2001 19:21', '14-jan-2002 10:00', '02-apr-2002 10:00',
         '18-jun-2002 13:00', '03-sep-2002 11:00', '18-nov-2002 10:00', '03-feb-2003 10:00',
         '22-apr-2003 12:00', '03-jul-2003 01:00', '06-oct-2003 00:00', '06-jan-2004 10:00',
         '04-apr-2004 10:00', '04-jul-2004 10:00', '03-oct-2004 10:00', '28-dec-2004 10:00',
         '31-mar-2005 10:00', '24-jun-2005 10:00', '23-sep-2005 10:00', '18-dec-2005 10:00',
         '18-mar-2006 10:00', '15-jun-2006 08:00', '16-sep-2006 13:00', '12-dec-2006 10:00',
	 '12-mar-2007 12:00', '27-may-2007 23:07', '01-sep-2007 09:43', '24-nov-2007 11:00',
	 '28-feb-2008 10:00', '22-may-2008 05:00', '20-aug-2008 15:00', '19-nov-2008 08:00',
	 '14-feb-2009 20:00', '08-may-2009 05:00', '12-aug-2009 02:00', '06-nov-2009 11:00',
	 '01-may-2010 10:00', '28-jul-2010 07:00', '16-feb-2011 10:00', '22-jul-2011 19:00',
	 '12-jan-2012 19:00', '09-jul-2012 03:00', '01-jul-2013 11:00', '30-dec-2013 04:00',
	 '24-jun-2014 20:00', '18-dec-2014 17:00', '11-jun-2015 23:20', '11-dec-2015 01:00',
	 '03-jun-2016 02:00', '24-may-2017 14:00', '14-may-2018 02:00', '06-may-2019 02:00',
	 '30-apr-2020 02:00', '15-apr-2021 15:00', '10-apr-2022 10:00', '28-mar-2023 10:00',
     '14-may-2024 08:00', '11-mar-2025 06:00']

In [4]:
print(len(on))
print(len(off))

86
86


In [5]:
monthDict = { "jan":1, "feb":2, "mar":3, "apr":4, "may":5, "jun":6, "jul":7, "aug":8, "sep":9, "oct":10, "nov":11, "dec":12}

In [6]:
heaterOnJD = np.zeros(len(on))
heaterOnDate = []
heaterOnTime = []

for i, bakeout in enumerate(on):
    date = bakeout.split(" ")[0]
    time = bakeout.split(" ")[1]
    heaterOnDate.append(date)
    heaterOnTime.append(time)

    d = int(date.split("-")[0])
    mstr = date.split("-")[1]
    m = monthDict[mstr]
    y = int(date.split("-")[2])

    h = int(time.split(":")[0])
    mm = int(time.split(":")[1])
    heaterOnJD[i] = np.round(astropy.time.Time({'year':y, 'month':m, 'day':d, 'hour':h, 'minute':mm}).jd,3)

heaterOffJD = np.zeros(len(off))
heaterOffDate = []
heaterOffTime = []

for i, bakeout in enumerate(off):
    date = bakeout.split(" ")[0]
    time = bakeout.split(" ")[1]
    heaterOffDate.append(date)
    heaterOffTime.append(time)

    d = int(date.split("-")[0])
    mstr = date.split("-")[1]
    m = monthDict[mstr]
    y = int(date.split("-")[2])

    h = int(time.split(":")[0])
    mm = int(time.split(":")[1])
    heaterOffJD[i] = np.round(astropy.time.Time({'year':y, 'month':m, 'day':d, 'hour':h, 'minute':mm}).jd,3)


In [7]:
combinedDates = []
combinedTimes = []
combinedStatuses = []
combinedJD = []

for i in range(len(on)):
    combinedStatuses.append('heaterOn')
    combinedDates.append(heaterOnDate[i])
    combinedTimes.append(heaterOnTime[i])
    combinedJD.append(heaterOnJD[i])
    
    combinedStatuses.append('heaterOff')
    combinedDates.append(heaterOffDate[i])
    combinedTimes.append(heaterOffTime[i])
    combinedJD.append(heaterOffJD[i])
    

In [8]:
t = Table([combinedStatuses, combinedDates, combinedTimes, combinedJD], names=('status','date_beg','time_beg','jd'))

In [10]:
t.write("./EIT/EITbakeoutDates_JD.txt",format="ascii",overwrite=True)

In [3]:
pseudobakeout_on = np.round(astropy.time.Time({'year':2023, 'month':12, 'day':29, 'hour':18, 'minute':0}).jd,3)
print(pseudobakeout_on)


2460308.25
